In [34]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np
from os.path import isfile
import torch.nn.functional as F
from data_handling.image_processing import PrepareData
from torch.utils.tensorboard import SummaryWriter

In [35]:
image_size = 128
credentials_location = ".gitignore/credentials_for_marketplace.yml"
images_folder = "data/cleaned_images_" + str(image_size)
X_array_folder = "data/X_for_image_"+ str(image_size) + ".npy"
y_array_folder = "data/y_for_image_"+ str(image_size)+".npy"



if isfile(X_array_folder) and isfile(y_array_folder): 
    X = np.load(X_array_folder)
    y = np.load(y_array_folder)
else:
    pipeline = PrepareData()
    product_df, image_df = pipeline.retrieve_dataframes(credentials_location)
    X, y = pipeline.form_arrays(images_folder, image_size)
    pipeline.save_to_files(X_array_folder, y_array_folder)



In [36]:
class ProductImageCategoryDataset(Dataset):
    def __init__(self, X, y, img_side_length):
        super().__init__()

        self.X = X
        self.y = y
        self.img_side_length = img_side_length
        
        assert len(self.X) == len(self.y)

    def __getitem__(self, index):
        features = self.X[index]
        label = self.y[index]
        features = torch.tensor(features)
        features = features.reshape(3, self.img_side_length, self.img_side_length)

        label = int(label)

        return (features, label)

    def __len__(self):
        return len(self.X)



In [37]:

dataset = ProductImageCategoryDataset(X,y, image_size)


In [38]:

train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

In [39]:
class CNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Conv2d(3, 8, 9),
            torch.nn.ReLU(),
            torch.nn.Conv2d(8, 16, 7),
            torch.nn.ReLU(),
            torch.nn.Flatten(),  # flatten
            torch.nn.Linear(207936, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 13),
            torch.nn.Softmax()  # turn into probabilities
        )

    def forward(self, features):
        return self.layers(features)



In [40]:
# Checks if GPU is avaliable to run on.
device = torch.device("cudo" if torch.cuda.is_available() else "cpu")

In [ ]:
learning_rate = 0.0001

In [42]:

model = CNN().to(device)
optimiser = torch.optim.Adam(model.parameters(), lr = learning_rate)
writer = SummaryWriter()

for batch_index, batch in enumerate(train_loader):

    features, labels = batch
    prediction = model(features.float())
    
    # calculate the loss
    loss = F.cross_entropy(prediction, labels)
    writer.add_scalar("Loss", loss.item(), batch_index)

    # calculate the gradients 
    loss.backward()

    #take an optimisation step
    optimiser.step()

    optimiser.zero_grad()
    




KeyboardInterrupt: 